# Imports

In [1]:
import tensorflow as tf

#para RaspberryPi
#import tflite_runtime.interpreter as tflite

import cv2
import numpy as np
import csv
import datetime

# Carga del modelo TFlite

In [2]:
#Path y nombre del modelo tflite
TFLITE_MODEL = "tflite_models/modelo_caras.tflite"
#En Rasp
#TFLITE_MODEL = "/usr/bin/modelo_caras.tflite"

tflite_interpreter = tf.lite.Interpreter(model_path=TFLITE_MODEL)
#En Rasp
#tflite_interpreter = tflite.Interpreter(model_path=TFLITE_MODEL)

tflite_interpreter.allocate_tensors()
input_details = tflite_interpreter.get_input_details()
output_details = tflite_interpreter.get_output_details()

# Configuracines

In [3]:
#permite utilizar la cámara
captura = cv2.VideoCapture(0)
#clasificador para buscar la posición de la cara
faceCascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
#En Rasp
#faceCascade = cv2.CascadeClassifier('/usr/share/opencv4/haarcascade_frontalface_default.xml')

#variable para el tiempo

# Predicciones

In [4]:
#Se crea el archivo donde se lleva el registro
file = open('predicciones.csv', 'a+', newline='')
with file:
    write = csv.writer(file)
    #Header
    write.writerows([['Prediccion','HORA']]) 

#Predicción ininterrumpida de las imágenes
while True:
    #se captura cada 5s
    cv2.waitKey(5000)
    
    #lectura de la imagen
    success,img = captura.read()
    
    #se requiere que imagen esté en escala de grises
    img_grises = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    
    #posiciones y detección de rostro(s)
    u = faceCascade.detectMultiScale(img_grises,1.1,4)
    
    #en caso de no ubicar una un rostro para que no se rompa el ciclo
    try:
        rostro = img[u[0][1]:u[0][1]+u[0][3],u[0][0]:u[0][0]+u[0][2]]
    except:
        break
        
    #se requiere cambiar el formato de la imagen para la predicción
    new_ima = np.float32(rostro)
    new_ima = cv2.resize(new_ima,(224,224))
    new_ima = np.expand_dims(new_ima,axis=0)
    new_ima = new_ima/255.0
    
    #predicción
    tflite_interpreter.set_tensor(input_details[0]['index'], new_ima)
    tflite_interpreter.invoke()
    #en esta variable está la predicción
    tflite_model_predictions = tflite_interpreter.get_tensor(output_details[0]['index'])
    #print de la predicción
    print(tflite_model_predictions)
    #para visualizar el rostro
    #no es necesario
    cv2.imshow('webcam',rostro)
    
    max_index = np.argmax(tflite_model_predictions)
    if max_index == 0:
        max_emotion="Enojo"
    if max_index == 1:
        max_emotion="Disgusto"
    if max_index == 2:
        max_emotion="Miedo"
    if max_index == 3:
        max_emotion="Alegre"
    if max_index == 4:
        max_emotion="Triste"
    if max_index == 5:
        max_emotion="Sorpresa"
        
    fecha = datetime.datetime.now()
    hora = datetime.datetime.strftime(fecha, '%H:%M:%S')
    
    file = open('predicciones.csv', 'a+', newline='')
    with file:
        write = csv.writer(file)
        write.writerows([[max_emotion,hora]])    

[[9.9417263e-01 3.7207433e-13 1.3871378e-17 1.0434601e-13 4.6312225e-16
  5.8273813e-03]]
[[9.0598762e-01 6.9794427e-11 4.3864406e-15 3.9004412e-12 5.9116427e-13
  9.4012454e-02]]
[[9.9993193e-01 6.7600455e-13 5.6084279e-15 4.2494389e-12 5.3348092e-13
  6.8028079e-05]]
[[6.4195085e-01 7.0184314e-10 1.2010240e-13 1.4310298e-11 3.4345304e-12
  3.5804918e-01]]
[[9.6228337e-01 4.0603207e-12 2.3835194e-15 2.9441942e-09 8.5098032e-14
  3.7716616e-02]]


KeyboardInterrupt: 